## Определения региона, где добыча принесёт наибольшую прибыль

<a href='#section1'>1. Загрузка и подготовка данных</a>

<a href='#section2'>2. Обучение и проверка модели</a>

<a href='#section3'>3. Подготовка к расчёту прибыли</a>

<a href='#section4'>4. Функция для расчёта прибыли</a>

<a href='#section5'>5. Расчёт прибыли и рисков </a>

<a id='section1'></a>
### 1. Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [2]:
NAME_1 = 'Первый регион'
NAME_2 = 'Второй регион'
NAME_3 = 'Третий регион'
d1 = pd.read_csv('/datasets/geo_data_0.csv')
d2 = pd.read_csv('/datasets/geo_data_1.csv')
d3 = pd.read_csv('/datasets/geo_data_2.csv')
d1.info()
d2.info()
d3.info()
d1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Все файлы содержат по 10 тысяч строк и по пять столбцов. Данные нужного типа, без пропусков, все числовые и примерно одного порядка. 

In [3]:
def split(df):
    features = df.drop(['product', 'id'], axis=1)
    target = df['product']
    return features, target

features1, target1 = split(d1)
features2, target2 = split(d2)
features3, target3 = split(d3)

Разделим выборки на целевой признак и остальные. id удалим, потому что он не нужен в дальнейшем для обучения модели.

<a id='section2'></a>
### 2. Обучение и проверка модели

Разделим данные на обучающую и валидационную выборки в соотношении 75:25.

Обучим модель и сделаем предсказания на валидационной выборке.

In [4]:
def learning(features, target, title):
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    rmse = np.sqrt(mean_squared_error(target_valid, predictions))     
    print()
    print(title)
    print('Размер обучающей выборки: ', features_train.shape[0])
    print('Размер валидационной выборки: ', features_valid.shape[0])
    print('Средний запас предсказанного сырья: ', np.mean(predictions))
    print('RMSE модели: ', rmse)
    return features_train, features_valid, target_train, target_valid, predictions
    

features_train1, features_valid1, target_train1, target_valid1, predictions1 = learning(features1, target1,
                                                                                   NAME_1)

features_train2, features_valid2, target_train2, target_valid2, predictions2 = learning(features2, target2,
                                                                                   NAME_2)

features_train3, features_valid3, target_train3, target_valid3, predictions3 = learning(features3, target3,
                                                                                   NAME_3)


Первый регион
Размер обучающей выборки:  75000
Размер валидационной выборки:  25000
Средний запас предсказанного сырья:  92.59256778438038
RMSE модели:  37.5794217150813

Второй регион
Размер обучающей выборки:  75000
Размер валидационной выборки:  25000
Средний запас предсказанного сырья:  68.728546895446
RMSE модели:  0.893099286775616

Третий регион
Размер обучающей выборки:  75000
Размер валидационной выборки:  25000
Средний запас предсказанного сырья:  94.96504596800489
RMSE модели:  40.02970873393434


В первом и третьем регионах примерно одинаковые средние запасы сырья и ошибка тоже приблизительно одинаковая. Во втором регионе ошибка модели совсем маленькая, а предсказанные запасы сырья примерно на треть меньше, чем в других регионах. 

<a id='section3'></a>
### 3. Подготовка к расчёту прибыли

Рассчитаем объём сырья достаточный для безубыточной разработки новой скважины.

In [5]:
TOTAL = 10000000000 # бюджет в регионе
PRICE = 450000 # прибыль от одной тысячи барелей
COUNT_RESEARCH = 500 # скважины для исследования
COUNT = 200 # скважины для разработки
PROBABILITY = 0.025 # максимальная вероятность убытков
QUANTILE = 0.95 # доверительный интервал
ZERO_PRICE = TOTAL / COUNT
print('Скважина должна приносить не менее ', ZERO_PRICE, ' прибыли')
print(round(ZERO_PRICE / PRICE), '- объём сырья достаточный для безубыточной разработки новой скважины')
print('Средние запасы в регионах:')
print('Первый', round(np.mean(predictions1)))
print('Второй', round(np.mean(predictions2)))
print('Третий', round(np.mean(predictions3)))

Скважина должна приносить не менее  50000000.0  прибыли
111 - объём сырья достаточный для безубыточной разработки новой скважины
Средние запасы в регионах:
Первый 93
Второй 69
Третий 95


Средние запасы в скважинах по первому и третьему региону немного меньше уровня безубыточности, а во втором регионе почти в два раза меньше. 

<a id='section4'></a>
### 4.  Функция для расчёта прибыли

Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [6]:
def best_total(predictions, target, title):
    df = target.to_frame()
    df['predict'] = predictions
    best = df.sort_values(by=['predict'], ascending = False).head(200)
    product = round(best['product'].sum()) # объём сырья
    result = round(best['product'].sum() * PRICE - TOTAL) # прибыль
    print()
    print(title)
    print('Объём сырья', product)
    print('Прибыль', result)
    
    return df, product, result

res1, product1, result1 = best_total(predictions1, target_valid1, NAME_1)
res2, product2, result2 = best_total(predictions2, target_valid2, NAME_2)
res3, product3, result3 = best_total(predictions3, target_valid3, NAME_3)


Первый регион
Объём сырья 29602
Прибыль 3320826043

Второй регион
Объём сырья 27589
Прибыль 2415086697

Третий регион
Объём сырья 28245
Прибыль 2710349964


Наибольшая прибыль в первом регионе, наименьшая - во втором.
<a id='section5'></a>
### 5. Расчёт прибыли и рисков для каждого региона

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков.

In [7]:
def best_values(df):
    best = df.sort_values(by=['predict'], ascending = False).head(200) 
    return round(best['product'].sum() * PRICE - TOTAL)

    
def region(df, title):
    state = np.random.RandomState(12345)
    values = []
    loss = 0
    for i in range(1000):
        subsample = df.sample(n = 500, replace = True, random_state = state)
        mean = best_values(subsample)
        if(mean < 0):
            loss += 1
        values.append(mean)
        
    values = pd.Series(values)
    quantile_from = round(values.quantile(0.025))
    quantile_to = round(values.quantile(QUANTILE + 0.025))
    print()
    print(title)
    print("Средняя прибыль:", round(values.mean()))
    print("95%-ый доверительный интервал: от", quantile_from, 'до', quantile_to)
    print('Риск убытков {:.1%}'.format(loss / 1000))


region(res1, NAME_1)   
region(res2, NAME_2)  
region(res3, NAME_3)  


Первый регион
Средняя прибыль: 396164985
95%-ый доверительный интервал: от -111215546 до 909766941
Риск убытков 6.9%

Второй регион
Средняя прибыль: 456045106
95%-ый доверительный интервал: от 33820509 до 852289454
Риск убытков 1.5%

Третий регион
Средняя прибыль: 404403867
95%-ый доверительный интервал: от -163350413 до 950359574
Риск убытков 7.6%


Для разработки скважин подходит только второй регион, т.к. только в этом регионе риск убытков меньше допустимого уровня, хотя запас сырья и прибыль меньше, чем в других.